# Base NN Architectures

In [1]:
import torch

In [13]:
from enum import Enum
import torch

from convnext_scaling.datasets import PhysionetDM, PhysioNet

In [7]:
logits = torch.randint(-10, 10, (10,)) * torch.rand(10)

In [10]:
logits.sigmoid()

tensor([0.3696, 0.9482, 0.9635, 0.1624, 0.7679, 0.1449, 0.0147, 0.7972, 0.7093,
        0.7887])

In [14]:
dataset = PhysioNet("/sc-scratch/sc-scratch-gbm-radiomics/ecg/physionet_challenge/training_pt/metadata_v5.csv")

In [101]:
mc = MetricCollection({
    "Recall": MultilabelRecall(num_labels=26),
    "Precision": MultilabelPrecision(num_labels=26),
    "AUROC": MultilabelAUROC(num_labels=26),
    "F1": MultilabelF1Score(num_labels=26),
    "AP": MultilabelAveragePrecision(num_labels=26),
    "Accuracy": MultilabelAccuracy(num_labels=26),
    "ECE": MultiLabelCalibrationError(num_labels=26)
})

for i in range(5):
    pred, gt = torch.rand(1, 26), torch.randint(0, 2, (1, 26))
    mc(pred, gt);

In [103]:
ece = MultiLabelCalibrationError(num_labels=26)
auroc = MultilabelAUROC(num_labels=26)

In [88]:
import matplotlib.pyplot as plt

In [98]:
import torch
from torchmetrics import Metric
from torchmetrics.functional import calibration_error

class MultiLabelCalibrationError(Metric):
    def __init__(self, num_labels: int, n_bins: int = 10, norm: str = "l1"):
        super().__init__()
        self.num_labels = num_labels
        self.n_bins = n_bins
        self.norm = norm

        # Separate lists to accumulate predictions and targets for each label
        self.add_state("preds", default=[], dist_reduce_fx="cat")
        self.add_state("targets", default=[], dist_reduce_fx="cat")

    def update(self, preds: torch.Tensor, targets: torch.Tensor):
        """
        Update metric states with predictions and targets.
        Assumes preds and targets are of shape (batch_size, num_labels).
        """
        if preds.shape[1] != self.num_labels or targets.shape[1] != self.num_labels:
            raise ValueError(f"Expected {self.num_labels} labels, but got {preds.shape[1]}.")

        self.preds.append(preds.detach().clone())
        self.targets.append(targets.detach().clone())

    def compute(self):
        """
        Compute calibration error for each label and return the results.
        """
        # Concatenate all batches
        preds = torch.cat(self.preds, dim=0)
        targets = torch.cat(self.targets, dim=0)

        labelwise_errors = []

        for i in range(self.num_labels):
            label_preds = preds[:, i]
            label_targets = targets[:, i]

            # Compute calibration error for this label
            ce = calibration_error(label_preds, label_targets, task="binary", n_bins=self.n_bins, norm=self.norm)
            labelwise_errors.append(ce)

        return torch.mean(torch.tensor(labelwise_errors))

In [112]:
import pandas as pd

In [115]:
weights = pd.read_csv("https://raw.githubusercontent.com/cph-cachet/LocalLeadAttention/refs/heads/main/weights.csv", index_col=0)
from torchmetrics.functional import confusion_matrix

In [122]:
def get_challenge_score(pred, gt):
    

In [137]:
import numpy as np

In [138]:
def compute_modified_confusion_matrix(labels, outputs):
    # Compute a binary multi-class, multi-label confusion matrix, where the rows
    # are the labels and the columns are the outputs.
    num_recordings, num_classes = np.shape(labels)
    A = np.zeros((num_classes, num_classes))

    # Iterate over all of the recordings.
    for i in range(num_recordings):
        # Calculate the number of positive labels and/or outputs.
        normalization = float(
            max(np.sum(np.any((labels[i, :], outputs[i, :]), axis=0)), 1))
        # Iterate over all of the classes.
        for j in range(num_classes):
            # Assign full and/or partial credit for each positive class.
            if labels[i, j]:
                for k in range(num_classes):
                    if outputs[i, k]:
                        A[j, k] += 1.0/normalization

    return A

In [ ]:
cm = compute_modified_confusion_matrix(pred, gt)

In [1]:
import torch
from torch import nn
from convnext_scaling.models.model_factory import MODELS
from convnext_scaling.train import train

[rank: 0] Seed set to 42


In [2]:
def count_params(model):
    return sum([x.numel() for x in model.parameters() if x.requires_grad])

@torch.no_grad()
def check_model(model, channels=12, seq_len=1024):
    """Runs one forward on random data."""
    _ = model(torch.rand(1, channels, seq_len))

In [3]:
print("MODEL NAME\t|\tPARAMETERS")
print("-" * 35)
for model_name, model_fn in MODELS.items():
    model = model_fn()
    params = count_params(model)
    print(f"{model_name}\t|\t{params:,}")
    check_model(model)

MODEL NAME	|	PARAMETERS
-----------------------------------
convnext_mini	|	13,383,098
convnext_tiny	|	26,787,770
convnext_small	|	48,132,026
convnext_base	|	85,458,842
convnext_large	|	192,036,698
resnet18	|	3,862,170
resnet50	|	16,012,442
resnet101	|	28,319,898
resnext18	|	12,867,482
resnext50	|	22,086,042
resnext101	|	79,676,826
vit_tiny	|	6,512,666
vit_small	|	25,616,922
vit_base	|	85,641,242
vit_large	|	303,054,362
getemed_small	|	952,770
getemed_base	|	3,160,770
getemed_large	|	13,558,210


# Dry run all models

In [4]:
path = "/sc-scratch/sc-scratch-gbm-radiomics/ecg/physionet_challenge/training_pt/metadata_v4.csv"
loss_fn = nn.BCEWithLogitsLoss()

In [5]:
for key in MODELS.keys():
    train(
        project="test",
        name="test_run_1",
        meta_file_path=path,
        fold=0,
        model_name=key,
        loss_fn=loss_fn,
        lr_decay_gamma=0.95,
        fast_dev_run=True,
    )

/home/jabareen/.conda/envs/ecg/lib/python3.12/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/jabareen/.conda/envs/ecg/lib/python3.12/site-p ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.
You are using a CUDA device ('NVIDIA A100-PCIE-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   

Epoch 0: 100%|██████████| 1/1 [00:00<00:00,  1.99it/s]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 0: 100%|██████████| 1/1 [00:00<00:00,  1.64it/s]                 

`Trainer.fit` stopped: `max_steps=1` reached.


Epoch 0: 100%|██████████| 1/1 [00:00<00:00,  1.64it/s]


/home/jabareen/.conda/envs/ecg/lib/python3.12/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/jabareen/.conda/envs/ecg/lib/python3.12/site-p ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type              | Params | Mode 
------------------------------------------------------
0 | model   | ConvNeXt          | 26.8 M | train
1 | loss_fn | BCEWithLogitsLoss | 0      | train
------------------------------------------------------
26.8 M    Trainable params
0         Non-trainable params
26.8 M    Total params
107.151   Total estimated model par

...

Epoch 0: 100%|██████████| 1/1 [00:00<00:00,  2.50it/s]


/home/jabareen/.conda/envs/ecg/lib/python3.12/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/jabareen/.conda/envs/ecg/lib/python3.12/site-p ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type                 | Params | Mode 
---------------------------------------------------------
0 | model   | ECGVisionTransformer | 13.6 M | train
1 | loss_fn | BCEWithLogitsLoss    | 0      | train
---------------------------------------------------------
13.6 M    Trainable params
0         Non-trainable params
13.6 M    Total params
54.233    Total esti

Epoch 0: 100%|██████████| 1/1 [00:00<00:00,  5.29it/s]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 0: 100%|██████████| 1/1 [00:00<00:00,  3.43it/s]                

`Trainer.fit` stopped: `max_steps=1` reached.


Epoch 0: 100%|██████████| 1/1 [00:00<00:00,  3.42it/s]
